- True implementation
  - LightGBM (scikit learn)
  - Loop: attack & train
    - annotate full-attacks dataset with costs
    - generate attacked datasets
    - continue training from previous forest with new data

In [8]:
import os
import pickle
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm
#import functools
#from os import listdir
#from os.path import isfile, join

In [10]:
DATASETS_PATH = "../data/census"
MODELS_PATH = "../out/models"
ATTACKER = "strong" # weak
TRAINING_SET="train_ori.csv.bz2" # original training set
TRAINING_SET_ATT="train_"+ATTACKER+"_att.csv.bz2" # perturbed training set
VALIDATION_SET="valid_ori.csv.bz2" # original validation set
VALIDATION_SET_ATT="valid_"+ATTACKER+"_att.csv.bz2" # perturbed validation set
TEST_SET="test_ori.csv.bz2" # original test set
TEST_SET_ATT="test_"+ATTACKER+"_att.csv.bz2" # perturbed test set

In [11]:
def load_dataset(path, dataset_filename, sep=","):
    return pd.read_csv(path+"/"+dataset_filename, sep=sep)

In [13]:
def infer_categorical_features(dataset):
    categorical_features = []
    for column in dataset.columns:
        if dataset[column].dtype == 'object':
            categorical_features.append(column)
    return categorical_features
            
def label_encode(dataset, categorical_features):
    dataset_le = dataset.copy()
    for column in dataset_le.columns:
        if column in categorical_features:
            dataset_le[column] = dataset_le[column].astype('category')
            dataset_le[column] = dataset_le[column].cat.codes
    return dataset_le

## Prepare Data

In [12]:
TRAIN = load_dataset(DATASETS_PATH, TRAINING_SET)
TRAIN_ATT = load_dataset(DATASETS_PATH, TRAINING_SET_ATT)

VALID = load_dataset(DATASETS_PATH, VALIDATION_SET)
VALID_ATT = load_dataset(DATASETS_PATH, VALIDATION_SET_ATT)

TEST = load_dataset(DATASETS_PATH, TEST_SET)
TEST_ATT = load_dataset(DATASETS_PATH, TEST_SET_ATT)

In [14]:
TRAIN_ATT_OFFSETS = TRAIN_ATT['instance_id'].value_counts().sort_index().values
VALID_ATT_OFFSETS = VALID_ATT['instance_id'].value_counts().sort_index().values
TEST_ATT_OFFSETS = TEST_ATT['instance_id'].value_counts().sort_index().values

In [15]:
def process_categorical_features(dataset):
    fx = infer_categorical_features(dataset)
    print("List of categorical features: [{}]"
          .format(", ".join([cf for cf in fx])))
    return label_encode(dataset, set(fx))

TRAIN = process_categorical_features(TRAIN)
TRAIN_ATT = process_categorical_features(TRAIN_ATT.iloc[:,1:])

VALID = process_categorical_features(VALID)
VALID_ATT = process_categorical_features(VALID_ATT.iloc[:,1:])

TEST = process_categorical_features(TEST)
TEST_ATT = process_categorical_features(TEST_ATT.iloc[:,1:])

List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]


## Load some model

In [18]:
def save_model(model_filename, model):
    with open(model_filename, 'wb') as fout:
        pickle.dump(model, fout)

In [19]:
def load_model(model_filename):
    with open(model_filename, 'rb') as fin:
        return pickle.load(fin)

In [23]:
std_model = load_model("../out/models/std_strong_200.pkl")

## Adversarial Boosting

In [94]:
def AdvBoosting_gen_data(model, data, groups):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    groups : grouping of same attacked instance 
    returns the new data matrix and new groups
    
    WARNING: currently works only for binary classification
    '''
    # score the datataset
    labels = data.iloc[:,-1]
    
    predictions = model.predict(data.iloc[:,:-1]) # exclude labels
    # binarize
    predictions = (predictions>0).astype(np.float)
    predictions = 2*predictions - 1
    
    # check mispredictions
    matchings = labels * predictions
    
    # select original data + attacked instances
    new_selected = [] # id of selected instances
    new_groups   = []
    
    offset = 0
    for g in groups:
        if g==1:
            # there are not attacks, just add original
            new_selected += [offset]
            new_groups   += [1]
        else:
            # get a slice of the matching scores
            g_matchings = matchings[offset:offset+g].values

            # most misclassified (smallest margin)
            # skip original
            adv_instance = np.argmin(g_matchings[1:])+1

            # add original and adversarial
            new_selected += [offset, adv_instance]
            new_groups   += [2]
        
        offset += g
    
    new_dataset = data.iloc[new_selected,:]
    
    return new_dataset, new_groups

In [95]:
def AdvBoosting_extend_model(model, data):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    returns the new model (is model modified inplace?)
    '''
    
    return None

In [93]:
adv_data, _ = AdvBoosting_gen_data(std_model, TRAIN_ATT, TRAIN_ATT_OFFSETS)

In [96]:
next_model = AdvBoosting_extend_model(std_model, adv_data)